## 1. Farmers' Market Expansion
<p>You have been hired by a farming organisation that helps local farmers sell their products. They want to know whether they should open up a new farmers' market to sell dairy products from nearby farmers. They have supplied you with daily shopping data from a panel of local households from 2019 to 2020. </p>
<p>The organization will make their decision based on whether dairy products are popular in the area, and whether sales are trending in a positive direction. To answer these questions, they want three pieces of data:</p>
<ol>
<li>What was the total number of purchases of dairy products for each month of 2020 (i.e., the <code>total_sales</code>)?</li>
<li>What was the total share of dairy products (out of all products purchased) for each month of 2020 (i.e., the <code>market_share</code>)?</li>
<li>For each month of 2020, what was the percentage increase or decrease in total monthly dairy purchases compared to the same month in 2019 (i.e., the <code>year_change</code>)?</li>
</ol>
<p>The organization handles not only dairy farmers, but also those with chicken farms. As a result, they are only interested in these three categories (which they treat as dairy): ‘whole milk’, 'yogurt' and 'domestic eggs'.</p>
<p>The data you need is available in the tables shown in the database schema below.</p>
<h5 id="databaseschema">Database Schema</h5>
<p><img src="https://assets.datacamp.com/production/repositories/5960/datasets/463543c8c38957ca5b95d93b02f2cb1bec53334f/diagram.PNG" alt="Database Schema" width="400px"></p>

In [119]:
%%sql
postgresql:///groceries

WITH dairy_sales_2020 AS(
    SELECT
        CAST(DATE_PART('month', CAST(fulldate AS DATE)) AS integer) AS month,
        COUNT(*) AS dairy_2020
    FROM purchases_2020
    INNER JOIN categories
    ON purchaseid = purchase_id
    WHERE LOWER(category) IN ('whole milk', 'yogurt', 'domestic eggs')
    GROUP BY month
),
market_sales_2020 AS(
    SELECT
        CAST(DATE_PART('month', CAST(fulldate AS DATE)) AS integer) AS month,
        COUNT(*) AS market_2020
    FROM purchases_2020
    INNER JOIN categories
    ON purchaseid = purchase_id
    GROUP BY month
),
dairy_sales_2019 AS(
    SELECT
        month,
        COUNT(*) AS dairy_2019
    FROM purchases_2019
    INNER JOIN categories
    USING (purchase_id)
    WHERE LOWER(category) IN ('whole milk', 'yogurt', 'domestic eggs')
    GROUP BY month
)
    
SELECT
    CAST(ds20.month AS integer),
    CAST(ds20.dairy_2020 AS integer) AS total_sales,
    ROUND(CAST(ds20.dairy_2020 AS NUMERIC)/ms.market_2020*100,2) AS market_share,
    ROUND((CAST(ds20.dairy_2020 AS NUMERIC) - ds19.dairy_2019)/ds19.dairy_2019*100,2) AS year_change
FROM dairy_sales_2020 AS ds20
INNER JOIN market_sales_2020 AS ms
USING (month)
INNER JOIN dairy_sales_2019 AS ds19
USING (month)
ORDER BY month;

12 rows affected.


month,total_sales,market_share,year_change
1,228,12.47,48.05
2,165,11.11,5.10
3,216,12.05,27.81
4,218,13.09,26.01
5,195,11.40,17.47
6,196,10.94,20.99
7,195,11.85,-0.51
8,231,11.77,32.00
9,180,11.16,4.65
10,193,11.61,17.68
